In [1]:
# Preliminary imports and auxiliary-function definition

from pathlib import Path
from devtools import debug
import tempfile

from fractal_server.app.runner.v2.models import Dataset, WorkflowTask
from fractal_server.app.runner.v2.runner import execute_tasks_v2
from fractal_server.images import SingleImage

from concurrent.futures import ThreadPoolExecutor
from image_data_exist_on_disk import image_data_exist_on_disk

import sys
sys.path.append("../tests/v2/04_runner")
from fractal_tasks_core_mock import TASK_LIST

executor = ThreadPoolExecutor()

{'name': 'create_ome_zarr_compound', 'input_types': {}, 'output_types': {}, 'function_non_parallel': <cyfunction create_ome_zarr at 0x731fa412e810>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x731fa412fac0>}
{'name': 'create_ome_zarr_multiplex_compound', 'input_types': {}, 'output_types': {}, 'function_non_parallel': <cyfunction create_ome_zarr_multiplex at 0x731fa412f100>, 'function_parallel': <cyfunction yokogawa_to_zarr at 0x731fa412fac0>}
{'name': 'MIP_compound', 'input_types': {'3D': True}, 'output_types': {'3D': False}, 'function_non_parallel': <cyfunction new_ome_zarr at 0x731fa416dd80>, 'function_parallel': <cyfunction maximum_intensity_projection at 0x731fa416e740>}
{'name': 'illumination_correction', 'input_types': {'illumination_correction': False}, 'output_types': {'illumination_correction': True}, 'function_non_parallel': None, 'function_parallel': <cyfunction illumination_correction at 0x731fa416c380>}
{'name': 'illumination_correction_compound', 'input_types':

In [2]:
# Create temporary directory for mocked zarrs
tmp_path = Path(tempfile.mkdtemp())
zarr_dir = (tmp_path / "zarr_dir").as_posix().rstrip("/")
print(f"{zarr_dir=}")

zarr_dir='/tmp/tmp4we222g0/zarr_dir'


In [3]:
# Run create_ome_zarr+yokogawa_to_zarr
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["create_ome_zarr_compound"],
            args_non_parallel=dict(image_dir="/tmp/input_images"),
        ),
    ],
    dataset=Dataset(zarr_dir=zarr_dir),
    executor=executor,
)

# Print current dataset information
debug(dataset);

[create_ome_zarr] START
[create_ome_zarr] image_dir='/tmp/input_images'
[create_ome_zarr] zarr_dir='/tmp/tmp4we222g0/zarr_dir'
[create_ome_zarr] zarr_path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr'
[create_ome_zarr] END
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr/A/01/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_01_0.tif'
[yokogawa_to_zarr] START
[yokogawa_to_zarr] path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr/A/02/0'
[yokogawa_to_zarr] raw_path='/tmp/input_images/A_02_0.tif'
[yokogawa_to_zarr] END
[yokogawa_to_zarr] END
/tmp/ipykernel_92975/3454169311.py:14 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
        ],
        zarr_dir='/tmp/tmp4we222g0/zarr_dir',
        images=[
            SingleImage(
                path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr/A/01/0',
                origin=None,
                attributes={
                    'well': 'A01',
              

In [4]:
# Run init_channel_parallelization
dataset = execute_tasks_v2(
    wf_task_list=[
        WorkflowTask(
            task=TASK_LIST["illumination_correction_compound"],
            args_non_parallel=dict(overwrite_input=False),
        ),
    ],
    dataset=dataset,
    executor=executor,
)

# Check that custom parallelization_list
assert len(dataset.images) == 4

for image in dataset.images[0:2]:
    assert image.types.get("illumination_correction") is None
for image in dataset.images[2:4]:
    assert image.types.get("illumination_correction") is True

# Print current dataset information
debug(dataset);

[illumination_correction_B] START
[illumination_correction_B] path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr/A/02/0_corr'
[illumination_correction_B] raw_path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr/A/02/0'
[illumination_correction_B] subsets={'C_index': 0}
[illumination_correction_B] path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr/A/02/0_corr'
[illumination_correction_B] START
[illumination_correction_B] path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr/A/02/0_corr'
[illumination_correction_B] raw_path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr/A/02/0'
[illumination_correction_B] subsets={'C_index': 1}
[illumination_correction_B] path='/tmp/tmp4we222g0/zarr_dir/my_plate.zarr/A/02/0_corr'
[illumination_correction_B] END
[illumination_correction_B] END
/tmp/ipykernel_92975/649487599.py:22 <module>
    dataset: Dataset(
        id=None,
        history=[
            'create_ome_zarr_compound',
            'illumination_correction_compound',
        ],
        zarr_dir='/tmp/tmp4we222g0/zarr_dir'

In [5]:
executor.shutdown()